In [1]:
import numpy as np
import time as tm
import os
import pandas as pd



# функция проверки транзакции.
def is_apple_sell(store, sku_num):
    return sku_num.startswith(store + '-ap')


# функция проверки транзакции.
def is_pen_sell(store, sku_num):
    return sku_num.startswith(store + '-pe')


def data_count(path_in, path_out):
    # агрегированные данные, вот 
    data = open(path_out + '-data.csv', 'w')
    data.write('year,state,apple_sold,apple_stolen,pen_sold,pen_stolen\n')
    for store in stores:
        steal = pd.read_csv(path_out + '/' + store + '-sell.csv',
                        sep = ',',
                        index_col = ['date'],
                        parse_dates = True)

# функция решения 1 и 2 задачи для заданного магазина.
def calc_calculation(path_in, path_out, store):
    # Фиксируем время начала работы ф-ии.
    start_time = tm.time()
    
    # Открываем файл, в который будем выводить состояние склада на каждый день.
    daily = open(path_out + '/' + store + '-daily.csv', 'w')
    daily.write('date,apple,pen\n')
    
    # Открываем файл, в который будем выводить кол-во сворованных яблок и карандашей.
    steal = open(path_out + '/' + store + '-steal.csv', 'w')
    steal.write('date,apple,pen\n')
    
    
    
    # Загружаем данные об инветаризации.
    inventory = pd.read_csv(path_in + '/' + store + '-inventory.csv',
                        sep = ',',
                        index_col = ['date'],
                        parse_dates = True)

    # Загружаем данные о продажах.
    sell = pd.read_csv(path_in + '/' + store + '-sell.csv',
                        sep = ',',
                        index_col = ['date'],
                        parse_dates = True)

    # Загружаем данные о поставках.
    supply = pd.read_csv(path_in + '/' + store + '-supply.csv',
                        sep = ',',
                        index_col = ['date'],
                        parse_dates = True)

    # Вычисляем, в какие года были собраны исходные данные.
    years = inventory.index.year.unique().tolist() + sell.index.year.unique().tolist() + supply.index.year.unique().tolist()
    years = np.sort(np.unique(years))

    # Текущее кол-во яблок на складе.
    apple = 0
    # Текущее кол-во карандашей на складе.
    pen = 0
    
    # Цикл по годам.
    for year in years:
        # Начинаем перебирать дни года year.
        time = pd.Timestamp(str(year))
        while time.year == year:
            # Обработка поставок в день time.
            for sup in supply[time:time].iterrows():        
                apple += sup[1].apple
                pen += sup[1].pen

            # Обработка продаж в день time.
            for transaction in sell[time:time].iterrows():
                if is_apple_sell(store, transaction[1].sku_num):
                    apple -= 1
                else:
                    pen -= 1

            # Если день time - последний день месяца, то...
            if time.is_month_end:

                # Вычисляем сколько яблок своровали.
                steal_apple = apple - inventory[time:time].iloc[0].apple
                apple -= steal_apple

                # Вычисляем сколько карандашей своровали.
                steal_pen = pen - inventory[time:time].iloc[0].pen
                pen -= steal_pen

                # Выводим данные о сворованных яблоках и карандашах.
                steal.write(time.strftime('%Y-%d-%m') + ',' + str(steal_apple) + ',' + str(steal_pen) + '\n')

            # Выводим данные о состоянии склада.
            daily.write(time.strftime('%Y-%d-%m') + ',' + str(apple) + ',' + str(pen) + '\n')

            # Переходим к следующему дню.
            time = time + pd.DateOffset(1)

    # Закрываем выходные файлы.
    daily.close()
    steal.close()
    
#подсчет агрегированных данных об объемах продаж и количестве сворованной продукции по штату и году    
def count_total_data(path_input, path_output, stores):
    start_time = tm.time()
    first = True
    state = stores[0][3]
    #создаем итоговую таблицу
    total_data = pd.DataFrame()
    for store in stores:
        if (store[3] != state):
            first = True
            state = store[3]
            if (total_data.empty):
                total_data = data
            else:
                #для каждого штата заносим данные в итоговую таблицу
                total_data = total_data.append(data)
        else:
            new_state = False
        
        #подсчет проданного по годам для одного магазина
        sold = pd.read_csv(path_input + '/' + store + '-sell.csv')
        sold['date'] = pd.to_datetime(sold['date'], infer_datetime_format=True)
        sold['date'] = sold['date'].map(lambda x: x.year)
        sell = pd.DataFrame()
        sell['apple_sold'] = sold[sold["sku_num"].map(lambda x: store+'-ap' in x)].groupby('date')['sku_num'].count()
        sell['pen_sold'] = sold[sold["sku_num"].map(lambda x: store+'-pe' in x)].groupby('date')['sku_num'].count()
        sell.index.names = ['year']
        
        #подсчет украденного по годам для одного магазина
        steal = pd.read_csv(path_output + '/' + store +'-steal.csv')
        steal['date'] = pd.to_datetime(steal['date'], format="%Y-%d-%m")
        steal['date'] = steal['date'].map(lambda x: x.year)
        steal = steal.groupby('date')['apple', 'pen'].sum()
        steal.index.names = ['year']
        steal.columns = ['apple_stolen', 'pen_stolen']
        
        #для каждого штата начинаем счет заново
        if (first == True):
            data = pd.concat([sell, steal], axis=1)
            data.insert(loc=0, column='state', value=store[3])
            data=data.reindex(columns=['state','apple_sold','apple_stolen','pen_sold','pen_stolen'])
            first = False
        else:
            data2 = pd.concat([sell, steal], axis=1)
            data2.insert(loc=0, column='state', value='\0')
            data2=data2.reindex(columns=['state','apple_sold','apple_stolen','pen_sold','pen_stolen'])
            data = data.add(data2, fill_value=0)
            
    #занесение данных в итоговую таблицу
    total_data = total_data.append(data)
    print(total_data)
    total_data.to_csv(path_output + '/' + 'total_data.csv')
    run_time = (tm.time() - start_time) / 60
    print('Агрегированные данные собраны за %.2f минут.' % (run_time))

   
path_infile = 'input'

path_outfile = 'answer'

# получаем список имен файлов
file_names = os.listdir(path_infile)

stores_names = []

# получаем имена магазинов
for i in file_names:
    first = i.find('-')
    second = i.find('-', first + 1)
    
    if second != -1:
        stores_names.append(i[:second]) 
    
stores_names = np.unique(stores)

# решение задачи 1 и 2 для всех магазинов.
for i in stores_names:
    calculation(path_infile, path_outfile, store_names)

#подсчет агрегированных данных об объемах продаж и количестве сворованной продукции по штату и году
count_total_data(path_input, path_output, stores)   

Магазин "MS-b1" обработан за 6.53 минут.
Магазин "MS-b2" обработан за 6.42 минут.
Магазин "MS-m1" обработан за 2.87 минут.
Магазин "MS-m2" обработан за 2.87 минут.
Магазин "MS-s1" обработан за 0.35 минут.
Магазин "MS-s2" обработан за 0.35 минут.
Магазин "MS-s3" обработан за 0.36 минут.
Магазин "MS-s4" обработан за 0.35 минут.
Магазин "MS-s5" обработан за 0.35 минут.
     state  apple_sold  apple_stolen  pen_sold  pen_stolen
year                                                      
2006     b     1362682           156    105110         182
2007     b     1362562           136    103878         152
2008     b     1370220           156    104064         146
2009     b     1364592           162    104822         192
2010     b     1360822           136    105200         192
2011     b     1366166           176    104206         200
2012     b     1370014           164    105156         182
2013     b     1366802           148    104266         180
2014     b     1364742           180    1